# Successful-Hollywood-Movies
## 1. Introduction
This project is based on a Competition I participated in through DataCamp. The problem statement was as follows: You have just been hired by a large movie studio to perform data analysis. Your manager, an executive at the company, wants to make new movies that "recapture the magic of old Hollywood." So you've decided to look at the most successful films that came out before Titanic in 1997 to identify patterns and help generate ideas that could turn into future successful films.

In this project I utilized SQL to extract ... 
and address the following questions:
1. How many movies are present in the database?
2. There seems to be a lot of missing data in the gross and budget columns. How many rows have missing data? What would you recommend your manager to do with these rows? 
3. How many different certifications or ratings are present in the database? 
4. What are the top five countries in terms of number of movies produced?
5. What is the average duration of English versus French movies? (Don't forget you can use the AI assistant!) 
6. Any other insights you found during your analysis? 

## 2. Data
The dataset containing Movies records was collected from [IMBD](https://www.imdb.com/).
The following table gives a brief description of the columns and key variables.
| Column name | Description |
|---|---|
| id | Unique movie identifier. |
| title | The title of the movie. |
| release_year | The year the movie was released to the public. |
| country | The country in which the movie was released. |
| duration | The runtime of the movie, in minutes. |
| language | The original language the movie was produced in. |
| certification | The [rating](https://simple.wikipedia.org/wiki/Motion_Picture_Association_film_rating_system) the movie was given based on their suitability for audiences. |
| gross | The revenue the movie generated at the box office, in USD. |
| budget | The available budget the production had for producing the movie, in USD. |

In [4]:
SELECT * 
FROM films;

SyntaxError: invalid syntax (1564898380.py, line 1)